* n-gram과 NNLM은 고정된 갯수의 단어만을 입력으로 받아야 하는 단점이 있음
* 그러나 시점(timestep)이라는 개념이 도입된 RNN으로 언어모델을 만든다면 입력의 길이를 고정 안해도 됨


### 1. RNN 언어모델
* RNNLM은 기본적으로 예측과정에서 이전 시점의 출력을 현재 시점의 입력
* 훈련과정에서는 이전 시점의 예측 결과를 다음 시점의 입력으로 넣으면서 예측하는 것이 아니라(테스트 과정에만 적용), 'what will the fact cat sit on' 이라는 학습샘플이 있다면, 'what will the fact cat sit' 시퀀스를 모델의 입력으로 넣으면, 'will the fact cat sit on'을 예측하도록 훈련함
* 이러한 방법을 **teacher forcing**이라고 하는데, 학습시 이 방법을 사용할 경우 모델이 t 시점에서 예측한 값을 t+1 시점에 입력으로 사용하지 않고, t 시점의 레이블, 즉 실제 알고 있는 정답을 t+1 시점의 입력으로 사용함
* 이렇게 하는 이유는 이전 시점의 출력 예측이 잘 못되면 뒤의 예측까지 영향을 미치기 때문임
----
* 학습 과정 동안 출력층에서 사용하는 활성화 함수는 **softmax** 함수임
* 손실함수는 'cross entropy' 함수 사용

----
#### RNNLM 구조 정리
* 총 4개의 층으로 구성
    * input layer : 현 시점의 입력 단어의 원-핫 벡터
    * Embedding layer(linear) : 단어 집합 크기가 V이고 임베딩 벡터의 크기를 M으로 설정하면 각 입력단어의 임베딩 층은 V x M 크기의 행렬 곱이 됨
    * Hidden layer(non-linear) : $h_{t} = tanh(W_{x} e_{t} + W_{h}h_{t−1} + b)$ 계산
    * Output layer : $\hat{y_{t}} = softmax(W_{y}h_{t} + b)$ 계산
    
    
* 룩업 테이블의 대상이 되는 테이블인 임베딩 행렬을 $E$라고 하였을 때, 결과적으로 RNNLM에서 학습 과정에서 학습되는 가중치 행렬은 다음의 $E$, $W_x$, $W_h$, $W_y$ 4개임

### 2. RNN을 이용한 텍스트 생성(Text Generation using RNN)

* 다대일 구조(many-to-one)의 RNN을 사용하여 텍스트 생성기 모델 만들기

#### 1) 데이터 이해 및 전처리

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [2]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.index_word) + 1
print("단어 집합 크기: %d" % vocab_size)

단어 집합 크기: 12


In [3]:
print(tokenizer.index_word)

{1: '말이', 2: '경마장에', 3: '있는', 4: '뛰고', 5: '있다', 6: '그의', 7: '법이다', 8: '가는', 9: '고와야', 10: '오는', 11: '곱다'}


In [12]:
### 학습데이터 만들기
sequences = list()
for line in text.split('\n'):
    encoded = tokenizer.texts_to_sequences([line])[0]  # 단어별로 인덱스 부여
    #print(len(encoded))
    for i in range(1, len(encoded)): 
        sequence = encoded[:i+1]  
        sequences.append(sequence)
        
print('학습에 사용할 샘플수: %d' % len(sequences))

학습에 사용할 샘플수: 11


In [13]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [19]:
## 크기가 다 다르기 때문에 샘플 길이를 가장 긴 샘플을 기준으로 통일
max_len = max(len(l) for l in sequences)

print('샘플의 최대 길이: {}'.format(max_len))

샘플의 최대 길이: 6


In [20]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

In [21]:
## X 데이터와 y 데이터 분리
X = sequences[:, :-1]
y = sequences[:, -1]
X, y

(array([[ 0,  0,  0,  0,  2],
        [ 0,  0,  0,  2,  3],
        [ 0,  0,  2,  3,  1],
        [ 0,  2,  3,  1,  4],
        [ 0,  0,  0,  0,  6],
        [ 0,  0,  0,  6,  1],
        [ 0,  0,  0,  0,  8],
        [ 0,  0,  0,  8,  1],
        [ 0,  0,  8,  1,  9],
        [ 0,  8,  1,  9, 10],
        [ 8,  1,  9, 10,  1]]),
 array([ 3,  1,  4,  5,  1,  7,  1,  9, 10,  1, 11]))

In [22]:
### one-hot encoding
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [24]:
y.shape, X.shape # y = 샘플수 * 단어집합 갯수(토큰수), 

((11, 12), (11, 5))

#### 2) 모델 설계 및 학습

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [26]:
# 임베딩 차원, 은닉상태 크기 설정, Fully Connected Layer
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))  # 12 * 10
model.add(SimpleRNN(hidden_units))  # hidden layer
model.add(Dense(vocab_size, activation='softmax')) # 활성화 함수 통과
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # y layer
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 2s - loss: 2.5026 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/200
1/1 - 0s - loss: 2.4910 - accuracy: 0.0000e+00 - 7ms/epoch - 7ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4796 - accuracy: 0.0909 - 5ms/epoch - 5ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4682 - accuracy: 0.0909 - 7ms/epoch - 7ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4567 - accuracy: 0.0909 - 4ms/epoch - 4ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4450 - accuracy: 0.1818 - 4ms/epoch - 4ms/step
Epoch 7/200
1/1 - 0s - loss: 2.4332 - accuracy: 0.1818 - 7ms/epoch - 7ms/step
Epoch 8/200
1/1 - 0s - loss: 2.4210 - accuracy: 0.1818 - 3ms/epoch - 3ms/step
Epoch 9/200
1/1 - 0s - loss: 2.4085 - accuracy: 0.2727 - 6ms/epoch - 6ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3955 - accuracy: 0.2727 - 3ms/epoch - 3ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3820 - accuracy: 0.2727 - 7ms/epoch - 7ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3679 - accuracy: 0.2727 - 4ms/epoch - 4ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3533 - accuracy: 0.3636

Epoch 105/200
1/1 - 0s - loss: 0.6751 - accuracy: 0.7273 - 5ms/epoch - 5ms/step
Epoch 106/200
1/1 - 0s - loss: 0.6639 - accuracy: 0.7273 - 4ms/epoch - 4ms/step
Epoch 107/200
1/1 - 0s - loss: 0.6527 - accuracy: 0.7273 - 8ms/epoch - 8ms/step
Epoch 108/200
1/1 - 0s - loss: 0.6416 - accuracy: 0.7273 - 5ms/epoch - 5ms/step
Epoch 109/200
1/1 - 0s - loss: 0.6307 - accuracy: 0.7273 - 6ms/epoch - 6ms/step
Epoch 110/200
1/1 - 0s - loss: 0.6198 - accuracy: 0.7273 - 6ms/epoch - 6ms/step
Epoch 111/200
1/1 - 0s - loss: 0.6091 - accuracy: 0.7273 - 6ms/epoch - 6ms/step
Epoch 112/200
1/1 - 0s - loss: 0.5985 - accuracy: 0.8182 - 6ms/epoch - 6ms/step
Epoch 113/200
1/1 - 0s - loss: 0.5880 - accuracy: 0.8182 - 5ms/epoch - 5ms/step
Epoch 114/200
1/1 - 0s - loss: 0.5776 - accuracy: 0.8182 - 7ms/epoch - 7ms/step
Epoch 115/200
1/1 - 0s - loss: 0.5673 - accuracy: 0.8182 - 8ms/epoch - 8ms/step
Epoch 116/200
1/1 - 0s - loss: 0.5572 - accuracy: 0.9091 - 5ms/epoch - 5ms/step
Epoch 117/200
1/1 - 0s - loss: 0.5472 - 

In [32]:
### 모델이 정확하게 예측하고 있는지 문장 생성 함수 만들기
def sentence_generation(model, tokenizer, current_word, max_len, n):
    
    init_word = current_word
    sentence = ""
    
    # n번 반복 : 반복횟수에 따라 예측 문장 길이가 달라짐
    for _ in range(n):
        # 전처리
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len, padding='pre')
        
        # 예측
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)
        
        # 예측한 인덱스에 대한 단어 추출
        for word, index in tokenizer.word_index.items():
            if index == result:  # result 값이 인덱스와 동일하다면 stop
                break
        
            # 현재 단어와 ' ' + 예측단어를 현재 단어로 변경
        current_word = current_word + ' ' + word
            
            # 예측단어를 문장에 저장
        sentence = sentence + ' ' + word
    
    sentence = init_word + sentence
    return sentence

In [33]:
print(sentence_generation(model, tokenizer, '경마장에', 5, 4))

경마장에 있는 말이 뛰고 있다


In [35]:
print(sentence_generation(model, tokenizer, '그의', 5, 2))

그의 말이 법이다


In [36]:
print(sentence_generation(model, tokenizer, '가는', 5, 5))  

가는 말이 고와야 오는 말이 곱다


### 3. LSTM을 이용하여 텍스트 생성하기

#### 1) 데이터에 대한 이해와 전처리

* 뉴욕타임즈 기사 제목 
https://www.kaggle.com/aashita/nyt-comments

In [37]:
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [38]:
df = pd.read_csv('ArticlesApril2018.csv')
df.tail()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
1319,5ae82c93068401528a2ab969,1004,By CLAIRE CAIN MILLER,article,This Common Question Reinforces the Gender Pay...,"['Discrimination', 'Wages and Salaries', 'Labo...",68,Upshot,3,2018-05-01 09:00:01,Unknown,Several states and cities have ordered employe...,The New York Times,News,https://www.nytimes.com/2018/05/01/upshot/how-...
1320,5ae82c95068401528a2ab96b,1043,By TRACY J. GATES,article,"Anna, Llama and Me","['Friendship', 'Dewdney, Anna', 'Writing and W...",65,Well,0,2018-05-01 09:00:02,Family,"The beginning, middle and end of a picture boo...",The New York Times,News,https://www.nytimes.com/2018/05/01/well/family...
1321,5ae82c9d068401528a2ab96d,659,Interview by AUDIE CORNISH,article,Gen. Michael Hayden Has One Regret: Russia,"['Classified Information and State Secrets', '...",66,Magazine,70,2018-05-01 09:00:06,Unknown,"The former N.S.A. and C.I.A. chief on Trump, S...",The New York Times,News,https://www.nytimes.com/2018/05/01/magazine/ge...
1322,5ae82c9f068401528a2ab96f,1155,By JASON ROBERT BROWN,article,There Is Nothin’ Like a Tune,"['Books and Literature', 'Purdum, Todd S', 'Th...",68,BookReview,17,2018-05-01 09:00:07,Book Review,"In “Something Wonderful,” Todd S. Purdum analy...",The New York Times,Review,https://www.nytimes.com/2018/05/01/books/revie...
1323,5ae82ca3068401528a2ab97a,2055,By MICHAEL PAULSON,article,Unknown,"['Theater', 'Tony Awards (Theater Awards)', 'A...",68,Culture,0,2018-05-01 09:00:09,Unknown,"A pair of two-part productions, “Harry Potter ...",The New York Times,News,https://www.nytimes.com/2018/05/01/theater/ton...


In [39]:
df.columns

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [40]:
df.shape

(1324, 15)

In [41]:
### headline 사용, null 값 확인
print(df.headline.isnull().sum())

0


In [45]:
## 헤드라인 리스트 작성
headlines = list(df.headline.values)
headlines[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [46]:
### Unknown 샘플 제거
headlines = [line for line in headlines if line != 'Unknown']
headlines[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [47]:
len(headlines)

1214

In [50]:
    # 구두점 제거, 소문자 변환
def repreprocessing(raw_sentence):
    preprocessed_sentence = raw_sentence.encode('utf8').decode('ascii', 'ignore')
    return ''.join(word for word in preprocessed_sentence if word not in punctuation).lower()

In [51]:
preprocessed_headlines = [repreprocessing(line) for line in headlines]
preprocessed_headlines[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [52]:
### 토큰 만들기
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headlines)
vocab_size = len(tokenizer.word_index) + 1
print('토큰 크기: %d' % vocab_size)

토큰 크기: 3494


In [57]:
### 정수인코딩 + 학습데이터 생성 : 모든 헤드라인에 대해 각 단어가 각 시점(time step)마다 하나씩 추가적으로 등장하는형태로 만듦
seqences = []

i = 0
for sequence in preprocessed_headlines:
    #print(sequence)
    encoded = tokenizer.texts_to_sequences([sequence])[0]
    #print(encoded)
    for j in range(1, len(encoded)):
        sequence = encoded[:j+1]
        #print(sequence)
        seqences.append(sequence)
        
seqences[:11]  # 첫번째 문장에 대한 정수인코딩 결과와 두번째 문장 시작

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [61]:
len(preprocessed_headlines[0].split(' '))

11

In [62]:
# 예측단어에 대해서도 정수인코딩

# index를 단어로 바꾸기
index_to_word = {}
for key, value in tokenizer.word_index.items():
    index_to_word[value] = key
    
print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [64]:
print('빈도수 상위 10번 단어 : {}'.format(index_to_word[10]))

빈도수 상위 10번 단어 : with


In [75]:
## 샘플 최대길이로 패딩 전처리
max_len = max(len(word) for word in seqences)  # 24
sequences = pad_sequences(seqences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [76]:
### X, y 데이터 분리
X = np.array(sequences)[:, :-1]
y = np.array(sequences)[:, -1]
X[:3]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,  99],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,  99, 269],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,  99, 269, 371]])

In [77]:
y[:3]

array([ 269,  371, 1115])

In [78]:
y = to_categorical(y, num_classes=vocab_size)

y[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

#### 2) 모델 설계하기

In [79]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [82]:
## 임베딩 벡터 차원은 10, 은닉상태 크기는 128, Fully Connected Layer을 출력층으로 하는 모델 설계
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))  # 3494 * 10
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 6s - loss: 7.6595 - accuracy: 0.0296 - 6s/epoch - 24ms/step
Epoch 2/200
244/244 - 4s - loss: 7.1170 - accuracy: 0.0308 - 4s/epoch - 17ms/step
Epoch 3/200
244/244 - 4s - loss: 6.9762 - accuracy: 0.0354 - 4s/epoch - 17ms/step
Epoch 4/200
244/244 - 4s - loss: 6.8463 - accuracy: 0.0415 - 4s/epoch - 17ms/step
Epoch 5/200
244/244 - 4s - loss: 6.6984 - accuracy: 0.0451 - 4s/epoch - 17ms/step
Epoch 6/200
244/244 - 4s - loss: 6.5354 - accuracy: 0.0486 - 4s/epoch - 18ms/step
Epoch 7/200
244/244 - 4s - loss: 6.3457 - accuracy: 0.0510 - 4s/epoch - 17ms/step
Epoch 8/200
244/244 - 4s - loss: 6.1409 - accuracy: 0.0563 - 4s/epoch - 17ms/step
Epoch 9/200
244/244 - 4s - loss: 5.9372 - accuracy: 0.0605 - 4s/epoch - 18ms/step
Epoch 10/200
244/244 - 4s - loss: 5.7447 - accuracy: 0.0687 - 4s/epoch - 17ms/step
Epoch 11/200
244/244 - 4s - loss: 5.5663 - accuracy: 0.0727 - 4s/epoch - 17ms/step
Epoch 12/200
244/244 - 4s - loss: 5.4006 - accuracy: 0.0764 - 4s/epoch - 17ms/step
Epoch 13/200


Epoch 100/200
244/244 - 4s - loss: 0.6295 - accuracy: 0.8765 - 4s/epoch - 17ms/step
Epoch 101/200
244/244 - 4s - loss: 0.6160 - accuracy: 0.8749 - 4s/epoch - 17ms/step
Epoch 102/200
244/244 - 4s - loss: 0.6019 - accuracy: 0.8813 - 4s/epoch - 17ms/step
Epoch 103/200
244/244 - 4s - loss: 0.5895 - accuracy: 0.8824 - 4s/epoch - 18ms/step
Epoch 104/200
244/244 - 4s - loss: 0.5783 - accuracy: 0.8847 - 4s/epoch - 17ms/step
Epoch 105/200
244/244 - 4s - loss: 0.5659 - accuracy: 0.8870 - 4s/epoch - 17ms/step
Epoch 106/200
244/244 - 4s - loss: 0.5545 - accuracy: 0.8872 - 4s/epoch - 18ms/step
Epoch 107/200
244/244 - 4s - loss: 0.5433 - accuracy: 0.8879 - 4s/epoch - 17ms/step
Epoch 108/200
244/244 - 4s - loss: 0.5348 - accuracy: 0.8904 - 4s/epoch - 18ms/step
Epoch 109/200
244/244 - 4s - loss: 0.5220 - accuracy: 0.8944 - 4s/epoch - 18ms/step
Epoch 110/200
244/244 - 4s - loss: 0.5126 - accuracy: 0.8947 - 4s/epoch - 18ms/step
Epoch 111/200
244/244 - 4s - loss: 0.5051 - accuracy: 0.8971 - 4s/epoch - 18

Epoch 198/200
244/244 - 5s - loss: 0.2628 - accuracy: 0.9161 - 5s/epoch - 19ms/step
Epoch 199/200
244/244 - 5s - loss: 0.2623 - accuracy: 0.9163 - 5s/epoch - 19ms/step
Epoch 200/200
244/244 - 5s - loss: 0.2648 - accuracy: 0.9155 - 5s/epoch - 21ms/step


In [81]:
vocab_size

3494

In [83]:
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ""
    
    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
        
        # 예측
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)
        
        for word, index in tokenizer.word_index.items():
            if index == result:
                break
                
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
        
    sentence = init_word + sentence
    return sentence

In [85]:
print(sentence_generation(model, tokenizer, 'i', 10))  # 10개 단어까지 예측

i cant jump ship from facebook yet russia into the wild


In [86]:
print(sentence_generation(model, tokenizer, 'how', 10)) 

how to make a crossword puzzle law in a bear and
